In [1]:
import numpy as np
import pandas as pd

## Reading and Writing Data in Text Format

In [3]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [4]:
pd.read_csv('examples/ex3.txt', sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [6]:
pd.read_csv('examples/ex4.csv')

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


The na_values option can take either a list or set of strings to consider missing values.

In [8]:
pd.read_csv('examples/ex5.csv', na_values=['NULL'])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different NA sentinels can be specified for each column in a dict:

In [10]:
sentinels = {'message': {'foo', 'NA'}, 'something': {'two'}}

pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


If you want to only read a small number of rows, specify that with nrows:

In [11]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read a file in pieces, specify a chunksize as a number of rows:

In [19]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

The TextFileReader object returned by read_csv allows you to iterate over the parts of the file according to the chunksize.

In [20]:
tot = pd.Series([])

for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format

In [23]:
import sys # To print the text result to the console

In [ ]:
data = pd.read_csv('examples/ex5.csv')

In [21]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [22]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [24]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [27]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'], na_rep='NULL')

a,b,c
1,2,3.0
5,6,NULL
9,10,11.0


### Working with Delimited Formats

For any file with a single-character delimiter, you can use Python's built-in csv module. Iterating through the reader like a file yields tuples of values with any quote characters removed:

In [30]:
import csv

with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
    
header, values = lines[0], lines[1:]

data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

To define a new format with a different delimiter, string quoting convention, or line terminator, we define a simple subclass of csv.Dialect:

In [32]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
    
# reader = csv.reader(f, dialect=my_dialect)

### XML and HTML: Web Scraping

In [33]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)

1

In [35]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [ ]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

root.INDICATOR returns a generator yielding each <INDICATOR> XML element. For each record, we can populate a dict of tag names to data values.

In [39]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)
    
perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


To handle an HTML link tag:

In [48]:
from io import StringIO

tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [49]:
root

<Element a at 0x264ab474c48>

In [50]:
root.get('href')

'http://www.google.com'

In [51]:
root.text

'Google'

## Binary Data Formats

pandas objects all have a to_pickle method that writes the data to disk in pickle format:

In [52]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [54]:
frame.to_pickle('examples/frame_pickle')

In [55]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### Using HDF5 Format

Compared with simpler formats, HDF5 supports on-the-fly compression with a variety of compression modes, enabling data with very large datasets that don't fit into memory, as you can efficiently read and write small sections of much larger arrays.

In [63]:
frame = pd.DataFrame({'a': np.random.randn(100)})

store = pd.HDFStore('mydata.h5')

store['obj1'] = frame

store['obj1_col'] = frame['a']

store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [59]:
store['obj1']

,a
0,-1.044205
1,-1.501867
2,-0.197638
3,-2.537831
4,0.956200
...,...
95,-0.727746
96,-3.237317
97,0.345827
98,-0.498118


HDFStore supports two storage schemas, 'fixed' and 'table'. The latter is generally slower, but it supports query operations using a special syntax:

In [64]:
store.put('obj2', frame, format='table')

store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,0.414838
11,-0.522917
12,-0.265005
13,1.223382
14,0.539913
15,1.064015


In [65]:
store.close()

The pandas.read_hdf function gives you a shortcut to these tools:

In [67]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.138799
1,-1.082464
2,-1.969013
3,-1.722725
4,0.906495


## Interacting with Web APIs

In [69]:
import requests 

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

resp = requests.get(url)
resp

<Response [200]>

In [71]:
data = resp.json()
data[0]['title']

'BUG: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.   import pandas.util.testing as tm'

In [72]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues.head()

,number,title,labels,state
0,33475,BUG: FutureWarning: pandas.util.testing is dep...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
1,33474,REF: unstack,[],open
2,33473,BUG: Series.__setitem__[listlike_of_integers] ...,[],open
3,33471,REF: Simplify __getitem__ by doing positional-...,[],open
4,33470,DOC: timezone conversion example added to pand...,[],open


## Interacting with Databases

In [ ]:
import sqlite3

In [73]:
query = """
CREATE TABLE TEST (
A VARCHAR(20),
B VARCHAR(20),
C REAL,
D INTEGER);"""

In [74]:
con = sqlite3.connect('mydata.sqlite')

In [75]:
con.execute(query)
con.commit()

In [81]:
data = [('Atlanta', 'Georgia', 1.25, 6),
       ('Talahassee', 'Florida', 2.6, 3),
       ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO TEST VALUES (?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [82]:
cursor = con.execute('SELECT * FROM TEST')

rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Talahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [83]:
cursor.description

(('A', None, None, None, None, None, None),
 ('B', None, None, None, None, None, None),
 ('C', None, None, None, None, None, None),
 ('D', None, None, None, None, None, None))

In [85]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,A,B,C,D
0,Atlanta,Georgia,1.25,6
1,Talahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [87]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')

pd.read_sql('SELECT * FROM TEST', db)

,A,B,C,D
0,Atlanta,Georgia,1.25,6
1,Talahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
